# Sub-section participation on a single Inversion Solution => NEW

In [11]:
#import pathlib
import json
import nzshm_model as nm
import geopandas as gpd
import solvis

from solvis.fault_system_solution_helper import FaultSystemSolutionHelper
from solvis_graphql_api.color_scale import ColourScaleNormaliseEnum, get_colour_values
from ipyleaflet import Map, GeoJSON

In [12]:
# load the composite_solution and shoose the Cruslal FaultSystemSolution
solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")  # Crustal 
helper = FaultSystemSolutionHelper(solution)

## Choose some faults and get their unique rupture IDs

In [13]:
# get ruptures passing through a named parent fault
TARGET_FAULTS = ['Awatere: Southwest', 'Pokeno'] #, 'Pokeno', 'BooBoo', "Masterton
ruptures = helper.ruptures_for_parent_fault_names(TARGET_FAULTS) # 

# get rupture fault sections (rs) with rates for those ruptures
df0 = solution.rs_with_rupture_rates
rupture_sections_df = df0[df0["Rupture Index"].isin(ruptures)]

In [14]:
section_count = len(rupture_sections_df['section'].unique())
print(f'the faults in {TARGET_FAULTS} have:')
print(f' {len(ruptures)} unique ruptures...')
print(f' {section_count} unique fault_sections...')
print()
section_ids = list(rupture_sections_df["Rupture Index"].unique())

the faults in ['Awatere: Southwest', 'Pokeno'] have:
 8 unique ruptures...
 123 unique fault_sections...



### A) Calculate sub-section participation rate (solvis)

In [23]:
# let's get the participation rate for each subsection in the rupture
subsections = [float(n) for n in helper.subsections_for_ruptures(section_ids)]
rate_column = "Annual Rate"
section_rates = solution.rs_with_rupture_rates[["Rupture Index", "section", rate_column]].groupby("section").agg('sum')[rate_column]
mro_sections_rates = section_rates[section_rates.index.isin(subsections)]
mro_sections_rates

section
3.0       0.000049
4.0       0.000049
19.0      0.001365
20.0      0.001365
21.0      0.001365
            ...   
2106.0    0.000485
2107.0    0.000485
2113.0    0.000302
2114.0    0.000302
2115.0    0.000302
Name: Annual Rate, Length: 123, dtype: float32

### B) Calculate sub-section participation rate (solvis-graphql-api)

In [16]:
# the following comes from SGI.cached.fault_section_aggregates_gdf, but is doctored slightly for IS column naming
fsr = solution.fault_sections_with_rupture_rates
fsr = fsr[fsr.section.isin(subsections)]
fsr = fsr.rename(columns={rate_column: "annual_rate"}) # the doctoring
section_aggregates = fsr.pivot_table(
        index=['section'],
        aggfunc=dict(annual_rate=['sum', 'min', 'max', 'mean'], Magnitude=['count', 'min', 'max', 'mean']),
    )
section_aggregates.columns = [".".join(a) for a in section_aggregates.columns.to_flat_index()]

# this is for illustration - showing the resulting rates are the same
section_aggregates['section_participation_rate'] = mro_sections_rates

### so we have the participation rate two ways
section_aggregates.drop(columns=['Magnitude.max', 'Magnitude.min', 'Magnitude.count', 'Magnitude.mean']).head()

,annual_rate.max,annual_rate.mean,annual_rate.min,annual_rate.sum,section_participation_rate
section,,,,,
3.0,0.000049,9.748369e-06,0.0,0.000049,0.000049
4.0,0.000049,9.748369e-06,0.0,0.000049,0.000049
19.0,0.000110,1.228274e-08,0.0,0.001365,0.001365
20.0,0.000110,1.207001e-08,0.0,0.001365,0.001365
21.0,0.000110,1.209537e-08,0.0,0.001365,0.001365


In [17]:
# join the fault sufaces (with geometry) and aggregated rates 
section_aggregates_detail = section_aggregates.join(solution.fault_surfaces(), 'section', how='inner', rsuffix='_R')
# section_aggregates_detail.head()

### Style the geojson using a color scale

In [18]:
# code from SGI.composite_solution.composite_rupture_sections.py 
fault_sections_gdf = gpd.GeoDataFrame(section_aggregates_detail)
color_values = get_colour_values(
                color_scale="inferno",
                color_scale_vmax=fault_sections_gdf['annual_rate.sum'].max(),
                color_scale_vmin=fault_sections_gdf['annual_rate.sum'].min(),
                color_scale_normalise= ColourScaleNormaliseEnum.LOG.value,
                values=tuple(fault_sections_gdf['annual_rate.sum'].tolist()),
            )
# print(color_values)
data = json.loads(fault_sections_gdf.to_json())

# merge the styling with the geojson
for feature, color in zip(data["features"], color_values):
    feature["properties"]["style"] = {
        "color": "#000000",
        "weight": 1,
        "fillColor": color,
        "fillOpacity": 1,
    }

### Display with ipyleaflet

In [19]:
center = [-41.5, 175]
zoom = 7
map = Map(center=center, zoom=zoom)
g = GeoJSON(data=data)
map.add(g)
map

Map(center=[-41.5, 175], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [20]:
# section_aggregates_detail[['section_participation_rate']]